In [8]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [12]:
DB_USER = "burnout_user"
DB_PASSWORD = "1234567890"
DB_HOST = "localhost"
DB_PORT = "3306"
DB_NAME = "burnoutdb"

In [3]:
db_config = {
  'user': DB_USER,
  'password': DB_PASSWORD,
  'host': 'localhost',
  'database': DB_NAME,
  'port': DB_PORT
}

# Инициализируем переменные, чтобы они были доступны в блоке finally
connection = None
cursor = None

try:
    # --- 2. Устанавливаем соединение ---
    print("Подключение к базе данных MySQL...")
    connection = mysql.connector.connect(**db_config)

    if connection.is_connected():
        print("Соединение с MySQL установлено.")

        # --- 3. Определяем SQL-запрос ---
        # Замените 'your_table' на имя вашей таблицы
        sql_query = "SELECT * FROM employees;"

        # --- 4. Выполняем запрос и загружаем данные в DataFrame ---
        # pd.read_sql() отлично работает с объектом соединения от mysql-connector
        print(f"Выполнение запроса: {sql_query}")
        df = pd.read_sql(sql_query, connection)

        # --- 5. Работаем с результатом ---
        print("Данные успешно загружены в Pandas DataFrame!")
        print("\nПервые 5 строк:")
        print(df.head())

        print("\nИнформация о DataFrame:")
        df.info()

# Обрабатываем возможные ошибки
except Error as e:
    print(f"Ошибка при работе с MySQL: {e}")

except Exception as e:
    print(f"Произошла непредвиденная ошибка: {e}")

finally:
    # --- 6. ОБЯЗАТЕЛЬНО закрываем соединение ---
    # Это освобождает ресурсы на сервере и в вашем приложении.
    if connection and connection.is_connected():
        connection.close()
        print("\nСоединение с MySQL закрыто.")

Подключение к базе данных MySQL...
Соединение с MySQL установлено.
Выполнение запроса: SELECT * FROM employees;
Данные успешно загружены в Pandas DataFrame!

Первые 5 строк:
   id  legal_entity gender                           city  \
0   1  Ñ„Ð¸Ð»Ð¸Ð°Ð»   None  Ð¡Ð°Ð½ÐºÑ‚-ÐŸÐµÑ‚ÐµÑ€Ð±ÑƒÑ€Ð³   
1   2  Ñ„Ð¸Ð»Ð¸Ð°Ð»   None  Ð¡Ð°Ð½ÐºÑ‚-ÐŸÐµÑ‚ÐµÑ€Ð±ÑƒÑ€Ð³   
2   3  Ñ„Ð¸Ð»Ð¸Ð°Ð»   None                   Ð¡Ð°Ð¼Ð°Ñ€Ð°   
3   4  Ñ„Ð¸Ð»Ð¸Ð°Ð»   None  Ð¡Ð°Ð½ÐºÑ‚-ÐŸÐµÑ‚ÐµÑ€Ð±ÑƒÑ€Ð³   
4   5  Ñ„Ð¸Ð»Ð¸Ð°Ð»   None  Ð¡Ð°Ð½ÐºÑ‚-ÐŸÐµÑ‚ÐµÑ€Ð±ÑƒÑ€Ð³   

                                            position  \
0                                               None   
1  ÐœÐµÐ½ÐµÐ´Ð¶ÐµÑ€ Ð¿Ð¾ Ñ€Ð°Ð±Ð¾Ñ‚Ðµ Ñ ÐºÐ»Ð¸Ðµ...   
2  Ð¡Ñ‚Ð°Ñ€ÑˆÐ¸Ð¹ Ð¼ÐµÐ½ÐµÐ´Ð¶ÐµÑ€ Ð³Ñ€ÑƒÐ¿Ð¿Ñ‹ Ñ...   
3  ÐœÐµÐ½ÐµÐ´Ð¶ÐµÑ€ Ð¿Ð¾ Ñ€Ð°Ð±Ð¾Ñ‚Ðµ Ñ ÐºÐ»Ð¸Ðµ...   
4  Ð¡Ñ‚Ð°Ñ€ÑˆÐ¸Ð¹ Ð¼ÐµÐ½ÐµÐ´Ð¶ÐµÑ€ Ð¿Ð¾ Ñ€Ð°Ð±Ð¾Ñ...   

                experience_text  age                 role_type kpi_june  \
0       5 Ð»ÐµÑ‚ 4 Ð¼ÐµÑÑÑ†Ð°   28  Ð ÑƒÐºÐ¾Ð

/var/folders/h8/sx47fdqd6xq2hht7cqw9mdvw0000gn/T/ipykernel_20514/141436597.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, connection)


In [13]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

db_config = {
  'user': DB_USER,
  'password': DB_PASSWORD,
  'host': 'localhost',
  'database': DB_NAME,
  'port': DB_PORT
}

# --- 1. Конфигурация подключения к вашей базе данных ---
def get_user_burnout_stats_v2(user_id: int, config: dict):
    """
    Получает 3 последние записи о тестах для пользователя,
    а также рассчитывает средние значения по всем его тестам за все время.

    Args:
        user_id (int): ID пользователя, для которого нужно получить данные.
        config (dict): Словарь с параметрами для подключения к БД.

    Returns:
        tuple: Кортеж, содержащий:
               - pandas.DataFrame: DataFrame с 3 последними записями.
               - dict: Словарь с общими средними значениями за все время.
        (None, None): если произошла ошибка или пользователь не найден.
    """
    connection = None
    try:
        connection = mysql.connector.connect(**config)
        
        # --- ЗАПРОС №1: Получаем 3 последние записи ---
        query_last_3 = """
        SELECT * 
        FROM burnout_tests 
        WHERE user_id = %s 
        ORDER BY test_datetime DESC 
        LIMIT 3;
        """
        df_last_3 = pd.read_sql(query_last_3, connection, params=[user_id])
        
        # Если DataFrame пуст, значит у пользователя нет записей. Выходим.
        if df_last_3.empty:
            print(f"Для пользователя с ID {user_id} записи не найдены.")
            return None, None

        # --- ЗАПРОС №2: Рассчитываем средние значения за ВСЕ время на стороне БД ---
        query_overall_avg = """
        SELECT
            AVG(score1) as avg_score1,
            AVG(score2) as avg_score2,
            AVG(score3) as avg_score3,
            AVG(burnout_score) as avg_burnout_score,
            AVG(mean_reaction_time_ms) as avg_reaction_time_ms
        FROM burnout_tests
        WHERE user_id = %s;
        """
        # pd.read_sql вернет DataFrame с одной строкой
        df_avg = pd.read_sql(query_overall_avg, connection, params=[user_id])
        
        # Преобразуем первую (и единственную) строку DataFrame в словарь
        overall_averages = df_avg.to_dict('records')[0]
        
        return df_last_3, overall_averages

    except Error as e:
        print(f"Ошибка при работе с MySQL: {e}")
        return None, None
    finally:
        if connection and connection.is_connected():
            connection.close()


# --- Основной блок для запуска (функция setup_database остается без изменений) ---
if __name__ == "__main__":
    # Если вы еще не создали таблицы, сохраните ваш SQL в файл 'schema.sql'
    # и раскомментируйте строку ниже для первого запуска.
    # from previous_script import setup_database # Предполагая, что функция в другом файле
    # setup_database(db_config)
    
    print("-" * 50)
    
    # Пример использования функции для пользователя с ID = 3
    target_user_id = 3
    print(f"Получаем данные для пользователя ID: {target_user_id}\n")
    
    last_3_records, overall_avg_scores = get_user_burnout_stats_v2(
        user_id=target_user_id, 
        config=db_config
    )
    
    # Проверяем, что данные получены
    if last_3_records is not None:
        print("ПОСЛЕДНИЕ 3 ЗАПИСИ:")
        print(last_3_records.to_string())
        
        print("\nОБЩИЕ СРЕДНИЕ ЗНАЧЕНИЯ (за все время):")
        if overall_avg_scores:
            for key, value in overall_avg_scores.items():
                print(f"{key}: {value:.2f}")

--------------------------------------------------
Получаем данные для пользователя ID: 3

ПОСЛЕДНИЕ 3 ЗАПИСИ:
    id  user_id       test_datetime  score1  score2  score3  burnout_score  mean_reaction_time_ms
0  503        3 2025-11-10 09:05:11      30      29      31           30.0                 670.98
1  453        3 2025-11-03 14:15:00      28      30      29           29.0                 650.45
2  403        3 2025-10-27 09:05:11      27      26      28           27.0                 630.98

ОБЩИЕ СРЕДНИЕ ЗНАЧЕНИЯ (за все время):
avg_score1: 26.82
avg_score2: 27.73
avg_score3: 28.91
avg_burnout_score: 27.82
avg_reaction_time_ms: 629.17


/var/folders/h8/sx47fdqd6xq2hht7cqw9mdvw0000gn/T/ipykernel_20514/2222105480.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_last_3 = pd.read_sql(query_last_3, connection, params=[user_id])
/var/folders/h8/sx47fdqd6xq2hht7cqw9mdvw0000gn/T/ipykernel_20514/2222105480.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_avg = pd.read_sql(query_overall_avg, connection, params=[user_id])
